<a href="https://colab.research.google.com/github/JoshuaXOng/Gunther/blob/develop/src/hand_signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
"""
I guess this file is for everything right now.

Author: Joshua Ong   
"""

import os
import torch
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np

class HSCNN(nn.Module):
    """ A CNN model for hand signs. """
    
    def __init__(self):
        super(HSCNN, self).__init__()
        # Tests...
        self.conv = nn.Conv2d(1, 1, 3, padding=1)
        self.max_pool = nn.MaxPool2d(3)
        self.fc_layer = nn.Linear(784, 24)

    def forward(Self, data):
        data = self.conv(data)
        data = self.max_pool(data)
        data = self.fc_layer(data)
        return data

class HSDataset(Dataset):
    """ Implements Dataset. 
    A custom dataset for hand-signals data.
    """

    def __init__(self, csv_file_path):
        """ Constructor.
        :param csv_file_path: the path to the csv file containing the dataset
            data. The file contains a header row; and, labels are in column 1 
            (1-indexing) and pixel data in the remaining columns up to 784.
        """
        data = np.loadtxt(csv_file_path, delimiter=",", skiprows=1, dtype=np.int)
        self.labels = data[:, 0]
        self.pixel_values = data[:, 1:]

    def __getitem__(self, index):
        return self.labels[index], self.pixel_values[index]

    def __len__(self):
        return len(self.labels)

In [ ]:
# Import dataset from local system 
# (relavant datasets are located in the data directory).
from google.colab import files
data = files.upload()
data2 = files.upload()

In [40]:
training_data = HSDataset("/content/sign_mnist_train.csv")
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

testing_data = HSDataset("/content/sign_mnist_test.csv")
testing_dataloader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [50]:
class Trainer():

    def __init__(self, no_epoch, model, training_loader, loss_fn, optimizer, device):
        
        self.no_epoch = no_epoch

        self.model = model
        self.training_loader = training_loader
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.device = device

        self.training_losses = []
        self.validation_losses = []
        self.accuracies = []

    def trainingDriver(self):

        for epoch in range(self.no_epoch): 
            train_loss = self._train(self.model, self.training_loader, self.loss_fn, self.optimizer, self.device)  
            training_losses.append(train_loss)
            print(train_loss)

    def _train(self, model, train_loader, loss_fn, optimizer, device):
        
        model.train()
        
        running_loss = 0
        
        for index, entry in enumerate(train_loader, 0): 
            
            inputs, labels = entry
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad() 
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() 

        return running_loss/len(train_loader)


In [ ]:
import torchvision.models as models
resnet18 = models.resnet18()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
resnet18.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=0.001)
test = Trainer(10, resnet18, training_dataloader, loss_fn, optimizer, device)
test.trainingDriver()
